<a href="https://colab.research.google.com/github/dataitgirls4/team_3/blob/miree/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC%2C%ED%85%90%EC%84%9C%ED%94%8C%EB%A1%9C%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EA%B0%90%EC%84%B1_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터열기

In [2]:
import pandas as pd
df = pd.read_csv('https://github.com/euphoris/datasets/raw/master/imdb.zip')

In [3]:
df.head()
#sentiment가 0이면 부정, 1이면 긍정

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


# 단어 문서 행렬 만들기

`CountVectorizer`를 사용하여 단어 문서 행렬을 만듭니다.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

- `max_features = 1000` : 피처의 개수는 1000개로 설정합니다.
- 영어 데이터이므로 `stop_words='english'`를 통해 관사 등을 제거합니다.

In [5]:
cv = CountVectorizer(max_features=1000, stop_words='english')

In [6]:
#단어별로 분할
tdm = cv.fit_transform(df['review'])

# 데이터 분할

`review`로 만든 단어 문서 행렬을 `x`로 `sentiment`를 `y`에 할당한다.

In [7]:
x=tdm
y=df['sentiment']

`x`와 `y`를 훈련용과 테스트용으로 분할한다. -> 나누지 않으면 훈련용이 과대평가될 수 있다.
- `test_size=0.2` : 20%를 무작위로 테스트용 데이터로 할당한다.
- `random_state` : 데이터를 할당할 때 생성할 난수의 씨앗값을 정한다. 재현가능한 결과를 위하여 하나의 숫자(예:42)로 고정한다.

In [8]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

`x_train` 은 `x`의 훈련용 데이터이다.

In [9]:
x_train.shape
#review의 개수가 800개

(800, 1000)

`x_test`는 `x`의 테스트용 데이터이다.

In [10]:
x_test

<200x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 928 stored elements in Compressed Sparse Row format>

`y_train`은 `y`의 훈련용 데이터이다.

In [11]:
y_train

29     1
535    0
695    1
557    0
836    0
      ..
106    1
270    1
860    1
435    1
102    0
Name: sentiment, Length: 800, dtype: int64

`y_test`는 `y`의 테스트용 데이터이다.

In [12]:
y_test

521    0
737    0
740    1
660    1
411    1
      ..
408    1
332    0
208    0
613    1
78     1
Name: sentiment, Length: 200, dtype: int64

# 저장

이후의 분석에 사용하기 위해 전처리된 데이터를 저장한다.

In [13]:
import joblib
joblib.dump((cv, x_train, x_test, y_train, y_test), 'imdb.pkl')

['imdb.pkl']

In [14]:
#cv저장이유: 단어 목록을 담아놔야 참고를 할 수 있기 때문에

# 텐서플로 설치

Tensorflow는 딥러닝 등 머신러닝을 위한 라이브러리이다. colab에서는 설치가 되어 있으므로 추가 설치가 필요없다. 아나콘다에서는 주피터 노트북에 다음과 같이 입력한다. (또는 !를 제외하고 명령창에 입력한다)

In [15]:
!pip install tensorflow

# 텐서플로를 이용한 감성 분석 실습

In [16]:
#데이터불러오기
cv, x_train, x_test, y_train, y_test = joblib.load('imdb.pkl')

# 로지스틱 회귀분석

In [17]:
import tensorflow as tf

In [18]:
model = tf.keras.models.Sequential()

In [19]:
model.add(tf.keras.layers.Dense(
    1,  #출력은 긍정(1)/부정(0) 하나
    input_shape = (1000,),  #입력은 단어 수
    activation='sigmoid'
))

모형 요약

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 1001      
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


- 학습 알고리즘(`optimizer`)는 경사하강법의 일종인 `adam`
- 손실함수(`loss`)는 교차엔트로피(`binary_crossentropy`)
- 보조적인 지표로 정확도(`accuracy`)를 사용한다. 정확도란 전체 사례 중 맞은 사례의 비율이다.

In [21]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 모형학습

- `batch_size`는 한 번에 데이터를 입력하는 크기로 여기서는 한 번에 32개의 데이터를 입력한다.
- 모든 데이터를 한 번 입력하는 것을 1 에포크(epoch)라고 한다. 아래는 `epochs=10`이므로 총 10에포크를 진행한다.

In [22]:
model.fit(x_train.toarray(), y_train.values, batch_size=32, epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6892 - accuracy: 0.5337
Epoch 2/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6778 - accuracy: 0.6175
Epoch 3/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6671 - accuracy: 0.6862
Epoch 4/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6568 - accuracy: 0.7325
Epoch 5/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6468 - accuracy: 0.7688
Epoch 6/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6368 - accuracy: 0.7962
Epoch 7/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6273 - accuracy: 0.8200
Epoch 8/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6181 - accuracy: 0.8438
Epoch 9/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6092 - accuracy: 0.8612
Epoch 10/10
25/25 [==============================] - 0s 1ms/step - loss: 0.6006 - accuracy: 0.8687


각 에포크마다 손실(`loss`)가 감소하고, 그에 따라 정확도(`accuracy`)가 증가하는 것을 볼 수 있다.

`ValueError: A Scipy sparse matrix was passed to a model that expects dense inputs. Please densify your inputs first, such as calling x.toarray()`와 같은 에러가 발생할 수 있다. 이때는 `x_train`을 `x_train.toarray()`로 바꿔준다.

# 모형 평가

테스트용 데이터로 모형을 평가한다.



In [23]:
model.evaluate(x_test.toarray(), y_test.values, verbose=0)


[0.649187445640564, 0.7400000095367432]

훈련 때보다 테스트에서 손실은 높고, 정확도는 낮다. 이를 통해 과적합이 발생하였음을 알 수 있다.



# 모형 저장


In [24]:
model.save('imdb-sentiment.krs')


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: imdb-sentiment.krs/assets
